In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score, \
roc_curve, auc, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth

In [17]:
movies = pd.read_csv('movies_classification_updated.csv')

#Preparing the data
y = movies['revenue']
X = movies.drop(["revenue", "title","Unnamed: 0"],axis = 1)
X.dropna(how = 'all', inplace = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [18]:
#Tuned AdaBoost model from Section 7.2.4
model_ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), random_state=1,n_estimators=25,learning_rate = 0.001).fit(X_train,y_train)
y_pred= model_ada.predict(X_test)
test_precision_ada = precision_score(y_test, y_pred)


# Tuned XGBoost model from Section 9.2.6
model_xgb = xgb.XGBClassifier(gamma= 0, learning_rate=0.25, max_depth=6, n_estimators= 150, reg_lambda=100, scale_pos_weight=1).fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)
test_precision_xgb = precision_score(y_test, y_pred)


# Tuned Decision tree classifier
model_dt = DecisionTreeClassifier(random_state = 1, max_depth= 9, max_features = 20, max_leaf_nodes = 140).fit(X_train, y_train)
y_pred = model_dt.predict(X_test)
test_precision_dt = precision_score(y_test, y_pred)


print("Adaboost model precision = ",test_precision_ada)
print("XGBoost model precision = ",test_precision_xgb)
print("Decision Tree Classifier model precision = ",test_precision_dt)

Adaboost model precision =  0.7895716945996276
XGBoost model precision =  0.7849740932642487
Decision Tree Classifier model precision =  0.7814685314685315


In [19]:
#Hard voting
ensemble_model = VotingClassifier(estimators=[('xgb',model_xgb),('ada',model_ada),('dt',model_dt)])
ensemble_model.fit(X_train,y_train)
precision_score(ensemble_model.predict(X_test), y_test)

0.6

In [13]:
#Soft Voting
ensemble_model = VotingClassifier(estimators=[('xgb',model_xgb),('ada',model_ada),('dt',model_dt)],
                                 voting='soft')
ensemble_model.fit(X_train,y_train)
precision_score(ensemble_model.predict(X_test), y_test)

0.6132596685082873

In [21]:
#The confusion matrix for the desired threshold for X_train
desired_threshold = 0.83

y_pred_prob = ensemble_model.predict_proba(X_train)[:,1] 

y_pred = y_pred_prob > desired_threshold
y_pred = y_pred.astype(int)

print("Accuracy: ",accuracy_score(y_pred, y_train)*100)  

fpr, tpr, auc_thresholds = roc_curve(y_train, y_pred_prob)
print("ROC-AUC: ",auc(fpr, tpr))# AUC of ROC

print("Precision: ", precision_score(y_train, y_pred))
print("Recall: ", recall_score(y_train, y_pred))

cm = pd.DataFrame(confusion_matrix(y_train, y_pred), 
                  columns=['Predicted 0', 'Predicted 1'], index = ['Actual 0', 'Actual 1'])
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g');

AttributeError: predict_proba is not available when voting='hard'